In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
 assembler = VectorAssembler(inputCols=['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                     'Length of Membership'],outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                351|
|   mean| 503.07926916526446|
| stddev|  77.74538969971488|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                149|
|   mean|  490.4442661227351|
| stddev|  82.48288580185401|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lr_model = lr.fit(train_data)

In [24]:
test_results = lr_model.evaluate(test_data)

In [25]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.076962011917317|
|  7.250995230153649|
|-12.678986394282163|
|  8.903052539120381|
| 0.7482414694836734|
| -4.910158011387523|
| 2.6565629838449354|
| -1.801314877679829|
| 17.965412554194245|
|  -5.74303588835329|
|  6.060673255638164|
|  -6.50984357465444|
|-10.538147955500563|
| -3.321091941277473|
| -18.63784606014707|
|   7.80010764036723|
| -2.581152644319218|
| -11.28948992023777|
|-17.214119770494847|
| 12.387671042197667|
+-------------------+
only showing top 20 rows



In [26]:
test_results.rootMeanSquaredError

10.324037555518299

In [27]:
test_results.r2

0.9842276643644998

In [28]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [33]:
unlabeled_data = test_data.select('features')

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1695067987115...|
|[31.3895854806643...|
|[31.5171218025062...|
|[31.5316044825729...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.9048571310136...|
|[31.9096268275227...|
+--------------------+
only showing top 20 rows



In [35]:
predictions = lr_model.transform(unlabeled_data)

In [36]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.0058318151109|
|[30.9716756438877...|487.38761452673907|
|[31.0662181616375...| 461.6122796019565|
|[31.1695067987115...| 418.4534782631724|
|[31.3895854806643...| 409.3213695904992|
|[31.5171218025062...|280.82857866177324|
|[31.5316044825729...| 433.8590427455176|
|[31.5761319713222...| 543.0278988670082|
|[31.6005122003032...| 461.2074389369027|
|[31.6253601348306...| 382.0799366452775|
|[31.6548096756927...|469.20275047191035|
|[31.7242025238451...|509.89773086261494|
|[31.8093003166791...| 547.3100473183417|
|[31.8124825597242...| 396.1314369250747|
|[31.8164283341993...| 519.7603375638034|
|[31.8209982016720...| 416.8751733728461|
|[31.8530748017465...| 461.8662761066712|
|[31.8648325480987...|451.18077039705145|
|[31.9048571310136...|  491.163977193311|
|[31.9096268275227...| 551.0583646310415|
+--------------------+------------